# This is a file for orbital rotation with a pre-computed unitary frame

In [50]:
import pandas as pd
import pyscf
from pyscf import gto, scf
from OrbitalRotation.tools.obt_to_tbt import get_obt_combined_tbt
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from basic_definitions import chemist_to_physicist
from pathlib import Path 

In [51]:
df = pd.read_csv("unitary_store/largest_unitary.csv")
U = df.values

In [52]:
iden = U @ U.conj().T
iden2 = U.conj().T @ U
print(np.allclose(iden, np.eye(U.shape[0])))
print(np.allclose(iden2, np.eye(U.shape[0])))

True
True


## load the original Hamiltonian with one-body term combined

In [53]:
mol = gto.M(atom="N 0 0 0; N 0 0 2.2", basis="sto3g", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)
print(f"n Cas: {ncas}")
print(f"orb sym: {orb_sym}")
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

obt_dummy = np.zeros((ncas, ncas))
combined_tbt = get_obt_combined_tbt(h1e, g2e, ncas, n_elec)

obt, tbt = chemist_to_physicist(obt_dummy, combined_tbt)

label = "_original"
filename = "fcidump.N2_22" + f"{label}"
pyscf.tools.fcidump.from_integrals(
    "fcidump_output"/Path(filename),
    obt,
    tbt,
    nmo=obt.shape[0],
    nelec=n_elec,
    nuc=ecore,
    ms=spin,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

obt_dummy = np.zeros((ncas, ncas))
combined_tbt = get_obt_combined_tbt(h1e, g2e, ncas, n_elec)

n Cas: 10
orb sym: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Apply the rotation and get the chemist tbt

In [54]:
p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, combined_tbt)[0]
rotated_tbt = np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, combined_tbt, optimize = p)

In [55]:
obt, tbt = chemist_to_physicist(obt_dummy, rotated_tbt)

In [56]:
label = "rotated"
filename = "fcidump.N2_22" + f"{label}"
pyscf.tools.fcidump.from_integrals(
    "fcidump_output"/Path(filename),
    obt,
    tbt,
    nmo=obt.shape[0],
    nelec=n_elec,
    nuc=ecore,
    ms=spin,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)